In [ ]:
# Code reference: 
# https://medium.com/@duyanhnguyen_38925/create-a-strong-text-classification-with-the-help-from-elmo-e90809ba29da

import pandas as pd

data_url = 'https://raw.githubusercontent.com/XinyanHe/413Dataset/master/train.csv'
data_df = pd.read_csv(data_url)
#import the dataset
data_df = data_df.head(1500)


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow.compat.v1 as tf1
import numpy as np

tf1.disable_eager_execution()
tf1.disable_v2_behavior()

#import the elmo model
elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)

#create an embedding layer for elmo
def elmo_embedding(x):
    return elmo(tf.squeeze(tf.cast(x,tf.string)),signature='default',as_dict=True)["default"]

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:

from keras.layers import Input, Lambda, Dense
from keras.models import Model

def build_model(): 
    input_text = Input(shape=(1,), dtype=tf.string)
    embedding = Lambda(elmo_embedding, output_shape=(1024, ))(input_text)
    dense = Dense(256, activation='relu')(embedding)
    pred = Dense(2, activation='softmax')(dense)
    model = Model(inputs=[input_text], outputs=pred)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
#build our classification model
model_elmo = build_model()
model_elmo.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
lambda (Lambda)              (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 262,914
Trainable params: 262,914
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn import metrics,preprocessing,model_selection
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical

X = data_df['DATA_COLUMN'].tolist()
y = data_df['LABEL_COLUMN'].tolist()
# split our data into train and test dataset
x_train, x_test, y_train, y_test = model_selection.train_test_split(np.asarray(X), np.asarray(y), test_size=0.2, random_state=42)
y_train = to_categorical(y_train)

In [ ]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    history = model_elmo.fit(x_train, y_train, epochs=2, batch_size=16)
    model_elmo.save_weights('./response-elmo-model.h5')

Train on 1200 samples
Epoch 1/2
1200/1200 [==============================] - 3504s 3s/sample - loss: 0.5304 - acc: 0.7425
Epoch 2/2
1200/1200 [==============================] - 2584s 2s/sample - loss: 0.4116 - acc: 0.8150


In [ ]:
with tf1.Session() as session:
    tf1.compat.v1.keras.backend.set_session(session)
    session.run(tf1.global_variables_initializer())  
    session.run(tf1.tables_initializer())
    model_elmo.load_weights('./response-elmo-model.h5')  
    predicts  = model_elmo.predict(x_test, batch_size=16)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


In [ ]:
print(metrics.confusion_matrix(y_test, predicts))

[[127  24]
 [ 23 126]]


In [ ]:
#get the result matrix 
print(metrics.classification_report(y_test, predicts))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84       151
           1       0.84      0.85      0.84       149

    accuracy                           0.84       300
   macro avg       0.84      0.84      0.84       300
weighted avg       0.84      0.84      0.84       300

